In [24]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone

import plotly.express as px
import joblib

In [25]:
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
df_nvidia_to_csv = yf.download("NVDA", start="2020-01-01", end="2025-08-13")


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'
nvidia_path = 'datasets/nvidia.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)
df_nvidia_to_csv.to_csv(nvidia_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)
df_nvidia = pd.read_csv(nvidia_path)

C:\Users\acer\AppData\Local\Temp\ipykernel_2488\126190450.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_2488\126190450.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_2488\126190450.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_2488\126190450.py:4: FutureWarning: YF.download() has changed argument auto_adjust de

In [26]:
df_apple.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,72.53851318359375,72.59889150198023,71.29230380764884,71.54588979671196,135480400
3,2020-01-03,71.83328247070312,72.59404771603535,71.6086769224392,71.76565905712691,146322800
4,2020-01-06,72.40567016601562,72.4443131708714,70.70300468370534,70.95418053006813,118387200


In [27]:
df_microsoft.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,152.7911376953125,152.89577671233235,150.61276208742146,151.04082551521063,22622100
3,2020-01-03,150.8886260986328,152.15380170810275,150.35592363465673,150.60326014654098,21116200
4,2020-01-06,151.2786407470703,151.34523581046366,148.8814652208115,149.4236895425469,20813700


In [28]:
df_amazon.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000
3,2020-01-03,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000
4,2020-01-06,95.14399719238281,95.18450164794922,93.0,93.0,81236000


In [29]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

df_nvidia['Date'] = df_nvidia['Price']
df_nvidia = df_nvidia.drop(index=[0,1], columns='Price')

In [30]:
df_apple.head()

,Close,High,Low,Open,Volume,Date
2,72.53851318359375,72.59889150198023,71.29230380764884,71.54588979671196,135480400,2020-01-02
3,71.83328247070312,72.59404771603535,71.6086769224392,71.76565905712691,146322800,2020-01-03
4,72.40567016601562,72.4443131708714,70.70300468370534,70.95418053006813,118387200,2020-01-06
5,72.06515502929688,72.67134838279743,71.84537704823941,72.41534488187352,108872000,2020-01-07
6,73.22442626953125,73.52631792855652,71.76810120120939,71.76810120120939,132079200,2020-01-08


In [31]:
df_microsoft.head()

,Close,High,Low,Open,Volume,Date
2,152.7911376953125,152.89577671233235,150.61276208742146,151.04082551521063,22622100,2020-01-02
3,150.8886260986328,152.15380170810275,150.35592363465673,150.60326014654098,21116200,2020-01-03
4,151.2786407470703,151.34523581046366,148.8814652208115,149.4236895425469,20813700,2020-01-06
5,149.89930725097656,151.8874339140462,149.65198528111424,151.55450216789137,21634100,2020-01-07
6,152.28695678710938,152.96235665110237,150.2512643514122,151.18349353765257,27746500,2020-01-08


In [32]:
df_amazon.head()

,Close,High,Low,Open,Volume,Date
2,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000,2020-01-02
3,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000,2020-01-03
4,95.14399719238281,95.18450164794922,93.0,93.0,81236000,2020-01-06
5,95.34300231933594,95.69450378417969,94.60199737548828,95.2249984741211,80898000,2020-01-07
6,94.59850311279297,95.55000305175781,94.3219985961914,94.9020004272461,70160000,2020-01-08


In [33]:
df_nvidia.head()

,Close,High,Low,Open,Volume,Date
2,5.971746444702148,5.971746444702148,5.892342616375051,5.942872368471369,237536000,2020-01-02
3,5.876163959503174,5.9199733524103735,5.8271273499510725,5.852019061256752,205384000,2020-01-03
4,5.900805473327637,5.906032689136935,5.756683394717106,5.782819473763594,262636000,2020-01-06
5,5.9722442626953125,6.018044830947195,5.884127936555174,5.929181691904879,314856000,2020-01-07
6,5.98344612121582,6.024766291068211,5.927937929074284,5.968013252371926,277108000,2020-01-08


In [34]:
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 2 to 1411
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   1410 non-null   object
 1   High    1410 non-null   object
 2   Low     1410 non-null   object
 3   Open    1410 non-null   object
 4   Volume  1410 non-null   object
 5   Date    1410 non-null   object
dtypes: object(6)
memory usage: 66.2+ KB


In [35]:
df_apple.describe()

,Close,High,Low,Open,Volume,Date
count,1410,1410,1410,1410,1410,1410
unique,1394,1410,1410,1410,1408,1410
top,139.26513671875,230.8000030517578,227.07000732421875,228.00999450683594,90956700,2025-08-12
freq,2,1,1,1,2,1


In [36]:
df_apple.isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
Date      0
dtype: int64

In [37]:
#INITIAL
dfs = [df_apple, df_amazon, df_microsoft, df_nvidia]
window = 14
short_period = 12
long_period = 26
signal_line_period = 9
years = list(range(2020,2026))
stocks = ['AAPL', 'AMZN', 'MSFT', 'NVDA']
X_train, X_test, y_train, y_test = None, None, None, None

In [38]:
#CONVERTING TO CORRECT FEATURE TYPES
def convert_type(df):
    df['Close'] = df['Close'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Open'] = df['Open'].astype(float)
    df['Volume'] = df['Volume'].astype(int)
    df['Date'] = pd.to_datetime(df['Date'])

In [39]:
#FEATURE ENGINEERING
def engineer_features(df):
    df['Price Change'] = df['Close'].diff().fillna(0)

    df['Gain'] = df.loc[df['Price Change'] > 0, 'Price Change']
    df['Loss'] = -df.loc[df['Price Change'] < 0, 'Price Change']

    df['Gain'] = df['Gain'].fillna(0)
    df['Loss'] = df['Loss'].fillna(0)

    df['Average Gain'] = df['Gain'].rolling(window=window).mean().fillna(0)
    df['Average Loss'] = df['Loss'].rolling(window=window).mean().fillna(0)

    df['RS'] = df['Average Gain'] / df['Average Loss']

    df['RSI'] = 100 - (100 / (1 + df['RS']))

    df['Short EMA'] = df['Close'].ewm(span=short_period,adjust=False).mean()
    df['Long EMA'] = df['Close'].ewm(span=long_period, adjust=False).mean()

    df['MACD'] = df['Short EMA'] - df['Long EMA']

    df['Signal Line'] = df['MACD'].ewm(span=signal_line_period, adjust=False).mean()
    
    df['MACD Histogram'] = df['MACD'] - df['Signal Line']
    
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    # df['Week'] = df['Date'].dt.weekday
    
    # df['Year-Month'] = df['Date'].dt.to_period('M').astype(str)
    #df['Week']
    return df

In [40]:
#DROPPING NULL AND UNIMPORTANT COLUMNS/ROWS
def drop_features_and_na(df):
    df = df.drop(columns=['Gain', 'Loss'])
    df = df.dropna()
    return df

In [41]:
#PRINTING SEABORN PLOTS
# def print_plots(df, stock):
#     for year in years:
#         sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='RSI', kind='line', height=7, aspect=2.74)
#         plt.title(f'RSI trend for year {year} Stock {stock}')
#         plt.savefig(f"plots/{stock}{year}.jpg", dpi=300, bbox_inches="tight")
#         # plt.show()
#         print(f'RSI trend for year {year} Stock {stock} downloaded!')
        
        # sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='MACD', kind='line')
        # plt.title(f'MACD trend for year {year} Stock {stock}')
        # plt.figure(figsize=(20,10))
        # plt.show()

# def print_plots(df, stock):
#     for year in years:
#         fig = px.line(data_frame=df.loc[df['Year'] == year], x='Date', y='RSI', title=f"RSI trend for year {year} Stock {stock}"f"RSI trend for year {year} Stock {stock}")
#         # plt.title(f'RSI trend for year {year} Stock {stock}')
#         # plt.savefig(f"plots/{stock}{year}.jpg", dpi=300, bbox_inches="tight")
#         fig.update_layout(width=1920, height=1080)
#         fig.write_html(f"../../frontend/public/{stock}{year}.html")
#         # plt.show()
#         print(f'RSI trend for year {year} Stock {stock} downloaded!')
        
#         # sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='MACD', kind='line')
#         # plt.title(f'MACD trend for year {year} Stock {stock}')
#         # plt.figure(figsize=(20,10))
#         # plt.show()

#1D INTERVAL
def print_1d_plot(df,stock):
    fig = px.line(data_frame=df, x='Date', y='Close')
    fig.update_layout(width=1920, height=1080)
    fig.write_html(f"../../frontend/public/{stock}1D.html")
    print(f'1 Day Interval trend for {stock}, downloaded!')

#TODO: 1W INTERVAL
def print_1w_plot(df,stock):
    fig = px.line(data_frame=df, x='Date', y='Close')
    fig.update_layout(width=1920, height=1080)
    fig.write_html(f"../../frontend/public/{stock}1W.html")
    print(f'1 Week Interval trend for {stock}, downloaded!')

#1M INTERVAL
def print_1m_plot(df,stock):
    fig = px.line(data_frame=df, x='Year-Month', y='Close')
    fig.update_layout(width=1920, height=1080)
    fig.write_html(f"../../frontend/public/{stock}1M.html")
    print(f'1 Month Interval trend for {stock}, downloaded!')

#1Y INTERVAL
def print_1y_plot(df,stock):
    fig = px.line(data_frame=df, x='Year', y='Close')
    fig.update_layout(width=1920, height=1080)
    fig.write_html(f"../../frontend/public/{stock}1Y.html")
    print(f'1 Year Interval trend for {stock}, downloaded!')

In [42]:
def print_plots(df,stock):
    print_1d_plot(df, stock)
    # print_1w_plot(df, stock)
    print_1m_plot(df, stock)
    print_1y_plot(df, stock)

In [43]:
#MODEL TRAINING AND EVALUATION
def split_data(df):
    X = df.drop(columns=['Date', 'Close'])
    y = df['Close']

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
def create_pipeline():
    model = clone(Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LinearRegression())
        ]))
    return model

In [45]:
def evaluate(model):
    kfold = KFold(n_splits=6, random_state=30, shuffle=True)

    params = {'lr__n_jobs': [10,20,50]}
    cv = clone(GridSearchCV(estimator=model,param_grid=params,cv=kfold))

    cv.fit(X_train, y_train)
    
    return {"best_params": cv.best_params_, "best_score": cv.best_score_}, cv

In [46]:
#MAIN OUTPUT
models = []

for i,df in enumerate(dfs):
    X_train, X_test, y_train, y_test = None, None, None, None
    
    convert_type(df)
    df = engineer_features(df)
    df = drop_features_and_na(df)
    
    X_train, X_test, y_train, y_test = split_data(df)
    
    model = create_pipeline()
    
    evaluation, cv = evaluate(model)
    df = df.drop(columns='Close')
    
    print("Best parameter:", evaluation['best_params']['lr__n_jobs'])
    print("Best score:", evaluation['best_score'])
    
    models.append({'model': cv, 'X_test': X_test,'y_test': y_test})
    
    print(f"{stocks[i]}:")
    print(df.head())
    print()
    
    # print_plots(df, stocks[i])

Best parameter: 10
Best score: 0.9996757910135172
AAPL:
         High        Low       Open     Volume       Date  Price Change  \
15  77.281835  76.634581  76.941301  101832400 2020-01-22      0.272911   
16  77.177993  76.233675  76.781915  104472000 2020-01-23      0.369522   
17  78.088477  76.685286  77.344620  146537600 2020-01-24     -0.222214   
18  75.296586  73.632564  74.883600  161940000 2020-01-27     -2.260559   
19  76.897814  75.398018  75.497040  162234000 2020-01-28      2.110817   

    Average Gain  Average Loss        RS        RSI  Short EMA   Long EMA  \
15      0.508211      0.208906  2.432720  70.868581  75.349469  74.270924   
16      0.534605      0.208906  2.559066  71.902738  75.618519  74.480359   
17      0.534605      0.174405  3.065305  75.401602  75.811991  74.657820   
18      0.493720      0.335874  1.469959  59.513490  75.627919  74.654687   
19      0.644493      0.311551  2.068659  67.412470  75.796907  74.808143   

        MACD  Signal Line  MAC

In [47]:
for i in range(len(models)):
    model_ = models[i]['model']
    X_test_ = models[i]['X_test']
    y_test_ = models[i]['y_test']
    
    y_pred_ = model_.predict(X_test_)
    
    print("MSE:", mean_squared_error(y_true=y_test_, y_pred=y_pred_))
    print("R2 Score:", r2_score(y_true=y_test_, y_pred=y_pred_))

    print(f"Predicted values for {stocks[i]}:")
    print(y_pred_[0:10])

    print(f"Correct values for {stocks[i]}:")
    print(y_test_[0:10])
    
    joblib.dump(model,f"models/{stocks[i]}.pkl")
    
    if joblib.load(f"models/{stocks[i]}.pkl"):
        print(f"{stocks[i]} model saved!")

MSE: 0.6584061486317561
R2 Score: 0.99964726598764
Predicted values for AAPL:
[133.87933153 226.14735644 229.23843821 190.085161   173.23488931
 151.81633014 141.83739708 213.61219575 206.69257354 192.05580122]
Correct values for AAPL:
378     133.879715
1280    227.203445
1234    229.056870
994     191.558197
493     172.296005
687     151.331192
708     141.161682
1407    213.008255
1354    206.625504
1003    191.974686
Name: Close, dtype: float64
AAPL model saved!
MSE: 0.7474608744933972
R2 Score: 0.9994624855857657
Predicted values for AMZN:
[172.70910923 236.81600251 198.19031053 144.54210475 169.73494761
 119.68112153 115.99454179 221.57567716 204.59276382 153.27502419]
Correct values for AMZN:
378     172.007996
1280    237.419998
1234    197.119995
994     145.889999
493     169.567505
687     118.540001
708     115.250000
1407    222.309998
1354    204.070007
1003    153.419998
Name: Close, dtype: float64
AMZN model saved!
MSE: 1.9403380736567577
R2 Score: 0.9997788685186785
P